In [1]:
!pwd

/kaggle/working


In [2]:
!pip install -q accelerate
!pip install -q transformers
!pip install --upgrade -q nltk
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q bert_score
!pip install -q mauve-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.3 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset, Dataset
from evaluate import load
import nltk
import os, torch
import pandas as pd

In [17]:
# Download NLTK resources if you haven't already
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
os.environ['HF_TOKEN'] = '<your-huggingface-token>'
os.environ["WANDB_DISABLED"] = "true"

In [6]:
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [8]:
EOS_TOKEN = gemma_tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN

'<eos>'

In [9]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        if input is None:
            input = ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [10]:
def process_dataset(dataset_name,
                    split_type,
                    processing_func,
                    rename_column = False,
                    filter_data = False,
                    filter_column_value = 'id',
                    filter_value = 'alpaca',
                    num_samples=20000):

    if isinstance(dataset_name, str):
        dataset = load_dataset(dataset_name, split=split_type)
    else:
        # Assuming dataset_name is a filepath for JSON file
        with open(dataset_name, 'r') as file:
            data = []
            for line_number, line in enumerate(file, 1):
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error parsing JSON at line {line_number}: {e}")
            dataset = Dataset.from_pandas(pd.DataFrame(data))

    if rename_column:
        dataset = rename(dataset)

    if filter_data:
        dataset = filter_dataset(dataset, num_samples, value, column_name)

    dataset = dataset.map(processing_func, batched=True)

    return dataset

# Define the additional processing steps
def rename(dataset):
    return dataset.rename_column('response', 'output')

def filter_dataset(dataset, num_samples, value, column_name):
    return dataset.filter(lambda example: value in example[column_name]).shuffle(seed=42).select(range(num_samples))


In [11]:
dataset_id = "HydraIndicLM/punjabi_alpaca_52K"

dataset = process_dataset(dataset_id, "train", formatting_prompts_func)


print(dataset['text'][0])

README.md:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]


### Instruction:
ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।

### Input:


### Response:
1. ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਖਾਓ ਅਤੇ ਬਹੁਤ ਸਾਰੇ ਫਲ ਅਤੇ ਸਬਜ਼ੀਆਂ ਨੂੰ ਸ਼ਾਮਲ ਕਰਨਾ ਯਕੀਨੀ ਬਣਾਓ।
2. ਆਪਣੇ ਸਰੀਰ ਨੂੰ ਕਿਰਿਆਸ਼ੀਲ ਅਤੇ ਮਜ਼ਬੂਤ ​​ਰੱਖਣ ਲਈ ਨਿਯਮਿਤ ਤੌਰ 'ਤੇ ਕਸਰਤ ਕਰੋ।
3. ਕਾਫ਼ੀ ਨੀਂਦ ਲਓ ਅਤੇ ਇਕਸਾਰ ਨੀਂਦ ਦਾ ਸਮਾਂ ਬਣਾਈ ਰੱਖੋ।<eos>


In [12]:
dataset

Dataset({
    features: ['input', 'instruction', 'output', 'text'],
    num_rows: 52002
})

In [13]:
print(dataset['text'][10])


### Instruction:
ਜੂਲੀਅਸ ਸੀਜ਼ਰ ਦੀ ਮੌਤ ਕਿਵੇਂ ਹੋਈ?

### Input:


### Response:
ਜੂਲੀਅਸ ਸੀਜ਼ਰ ਦੀ ਹੱਤਿਆ 60 ਤੱਕ ਸਾਜ਼ਿਸ਼ਕਾਰਾਂ ਦੇ ਇੱਕ ਸਮੂਹ ਦੁਆਰਾ ਕੀਤੀ ਗਈ ਸੀ, ਜਿਸ ਦੀ ਅਗਵਾਈ ਗੇਅਸ ਕੈਸੀਅਸ ਲੌਂਗੀਨਸ ਅਤੇ ਮਾਰਕਸ ਜੂਨੀਅਸ ਬਰੂਟਸ, ਸੀਨੇਟ ਹਾਊਸ ਵਿੱਚ ਮਾਰਚ (15 ਮਾਰਚ) 44 ਈਸਵੀ ਪੂਰਵ ਦੇ ਆਈਡਸ ਵਿੱਚ ਕੀਤੀ ਗਈ ਸੀ।<eos>


In [14]:
df = dataset.to_pandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52002 entries, 0 to 52001
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input        52002 non-null  object
 1   instruction  52002 non-null  object
 2   output       52002 non-null  object
 3   text         52002 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [15]:
df.to_csv("dataset.csv")

In [16]:
df.head()

,input,instruction,output,text
0,,ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।,1. ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਖਾਓ ਅਤੇ ਬਹੁਤ ਸਾਰੇ ਫਲ ਅਤੇ ਸਬਜ਼...,\n### Instruction:\nਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ...
1,,ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਕੀ ਹਨ?,"ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਲਾਲ, ਨੀਲੇ ਅਤੇ ਪੀਲੇ ਹਨ।",\n### Instruction:\nਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਕੀ ਹਨ?\n...
2,,ਪਰਮਾਣੂ ਦੀ ਬਣਤਰ ਦਾ ਵਰਣਨ ਕਰੋ।,"ਇੱਕ ਪਰਮਾਣੂ ਇੱਕ ਨਿਊਕਲੀਅਸ ਦਾ ਬਣਿਆ ਹੁੰਦਾ ਹੈ, ਜਿਸ ...",\n### Instruction:\nਪਰਮਾਣੂ ਦੀ ਬਣਤਰ ਦਾ ਵਰਣਨ ਕਰੋ...
3,,ਅਸੀਂ ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਕਿਵੇਂ ਘਟਾ ਸਕਦੇ ਹਾਂ?,"ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਘਟਾਉਣ ਦੇ ਕਈ ਤਰੀਕੇ ਹਨ, ਜਿਵੇਂ ਕ...",\n### Instruction:\nਅਸੀਂ ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਕਿਵੇ...
4,,ਉਸ ਸਮੇਂ ਦਾ ਵਰਣਨ ਕਰੋ ਜਦੋਂ ਤੁਹਾਨੂੰ ਇੱਕ ਮੁਸ਼ਕਲ ਫੈ...,ਜਦੋਂ ਮੈਂ ਇੱਕ ਉਸਾਰੀ ਕੰਪਨੀ ਵਿੱਚ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਮੈਨ...,\n### Instruction:\nਉਸ ਸਮੇਂ ਦਾ ਵਰਣਨ ਕਰੋ ਜਦੋਂ ਤ...


In [28]:
#test_dataset_obj = dataset.select(range(47000,52002))
test_dataset_obj = dataset.select(range(47000,52002))
test_dataset_obj

Dataset({
    features: ['input', 'instruction', 'output', 'text'],
    num_rows: 5002
})

In [18]:
test_dataset_obj[4]

{'input': 'ਇਸ ਵਾਕ ਵਿੱਚ ਕੋਈ ਗਲਤੀ ਨਹੀਂ ਹੈ।',
 'instruction': "ਜਾਂਚ ਕਰੋ ਕਿ ਕੀ ਇਹ ਵਾਕ ਵਿਆਕਰਨਿਕ ਤੌਰ 'ਤੇ ਸਹੀ ਹੈ। ਆਉਟਪੁੱਟ ਹਾਂ ਜਾਂ ਨਹੀਂ।",
 'output': 'ਹਾਂ',
 'text': "\n### Instruction:\nਜਾਂਚ ਕਰੋ ਕਿ ਕੀ ਇਹ ਵਾਕ ਵਿਆਕਰਨਿਕ ਤੌਰ 'ਤੇ ਸਹੀ ਹੈ। ਆਉਟਪੁੱਟ ਹਾਂ ਜਾਂ ਨਹੀਂ।\n\n### Input:\nਇਸ ਵਾਕ ਵਿੱਚ ਕੋਈ ਗਲਤੀ ਨਹੀਂ ਹੈ।\n\n### Response:\nਹਾਂ<eos>"}

In [19]:
base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    #"google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [30]:
#test_dataset_obj[924:925]
#test_dataset_obj = test_dataset_obj.select(range(924, 5002))
test_dataset_obj = test_dataset_obj.select(range(4286, 5002))
test_dataset_obj

Dataset({
    features: ['input', 'instruction', 'output', 'text'],
    num_rows: 4078
})

In [31]:
import csv

In [ ]:
predictions = []
references = []
count = 923
for example in test_dataset_obj:
    instructions = example["instruction"]
    inputs = example["input"]
    
    text = alpaca_prompt.format(instructions, inputs, "")
    reference = example["output"]  # the desired output
    
    inputs_tokenized = gemma_tokenizer([text], return_tensors = "pt").to("cuda")
    
    outputs = base_model.generate(**inputs_tokenized, max_new_tokens = 300)
    decoded_outputs = gemma_tokenizer.batch_decode(outputs)
    generated_text = decoded_outputs[0]
    #print(generated_text)
    #print("=======")

    predictions.append(generated_text[5:-5])
    references.append(reference)

    with open("base_preds4.csv", 'a+', newline='') as csvfile:
        writer = csv.writer(csvfile)
    
        # Write each row to the CSV file
        #for row in data:
        writer.writerow([generated_text[5:-5], reference])
    count += 1
    print("Count:", count, end=", ")
    
    #count += 1
    #if count == 5:
    #    break

In [7]:
base_evaluation_df = pd.DataFrame()
base_evaluation_df['base_preds'] = predictions
base_evaluation_df['base_references'] = references
base_evaluation_df.to_csv("base_preds2.csv", index=None)

NameError: name 'predictions' is not defined

In [1]:
import pandas as pd
saved_preds_df = pd.read_csv("base_preds4.csv")
saved_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 2 columns):
 #   Column                                                                                                                                                                                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                                                                                  --------------  ----- 
 0   
### Instruction:
ਪੰਜ ਸਵਾਲ ਤਿਆਰ ਕਰੋ ਜੋ ਲੋਕ ਆਪਣੇ ਕਰੀਅਰ ਦੇ ਉਦੇਸ਼ ਨੂੰ ਖੋਜਣ ਲਈ ਆਪਣੇ ਆਪ ਨੂੰ ਪੁੱਛ ਸਕਦੇ ਹਨ

### Input:


### Response:
                                                                                                                                                 

In [2]:
saved_preds_df.head()

,\n### Instruction:\nਪੰਜ ਸਵਾਲ ਤਿਆਰ ਕਰੋ ਜੋ ਲੋਕ ਆਪਣੇ ਕਰੀਅਰ ਦੇ ਉਦੇਸ਼ ਨੂੰ ਖੋਜਣ ਲਈ ਆਪਣੇ ਆਪ ਨੂੰ ਪੁੱਛ ਸਕਦੇ ਹਨ\n\n### Input:\n\n\n### Response:\n,1. ਕਿਹੜੀ ਚੀਜ਼ ਮੈਨੂੰ ਸਭ ਤੋਂ ਵੱਧ ਪ੍ਰੇਰਿਤ ਅਤੇ ਪ੍ਰੇਰਿਤ ਮਹਿਸੂਸ ਕਰਦੀ ਹੈ?\n2. ਮੈਨੂੰ ਕਿਹੜੀਆਂ ਗਤੀਵਿਧੀਆਂ ਦਾ ਸਭ ਤੋਂ ਵੱਧ ਆਨੰਦ ਆਉਂਦਾ ਹੈ?\n3. ਮੇਰੇ ਜਨੂੰਨ ਅਤੇ ਮੂਲ ਮੁੱਲ ਕੀ ਹਨ?\n4. ਮੈਂ ਕੁਦਰਤੀ ਤੌਰ 'ਤੇ ਕਿਸ ਚੀਜ਼ ਵਿੱਚ ਪ੍ਰਤਿਭਾਸ਼ਾਲੀ ਹਾਂ?\n5. ਮੈਂ ਕਿਹੜੀਆਂ ਸਮੱਸਿਆਵਾਂ ਨੂੰ ਹੱਲ ਕਰਨ ਲਈ ਸਭ ਤੋਂ ਵੱਧ ਮਹਿਸੂਸ ਕਰਦਾ ਹਾਂ?
0,\n### Instruction:\n5 ਬਾਹਰੀ ਗਤੀਵਿਧੀਆਂ ਦੀ ਸੂਚੀ ...,1. ਕੁਦਰਤ ਦੀ ਸੈਰ ਕਰੋ।\n2. ਸਾਈਕਲ ਦੀ ਸਵਾਰੀ ਲਈ ਜਾਓ...
1,\n### Instruction:\nਪੰਜ ਗੁਣ ਦੱਸੋ ਜੋ ਕਿਸੇ ਨੂੰ ਪ...,1. ਭਰੋਸੇਯੋਗ\n2. ਉਦਾਰ\n3. ਸਹਾਇਕ\n4. ਗੈਰ-ਨਿਰਣਾਇਕ...
2,\n### Instruction:\nਇੱਕ ਕਾਲਪਨਿਕ ਸੌਫਟਵੇਅਰ ਕੰਪਨੀ...,ਅੰਤਮ ਸੌਫਟਵੇਅਰ ਪੈਕੇਜ ਬਣਾਓ ਅਤੇ 30% ਦੀ ਛੂਟ ਪ੍ਰਾਪਤ...
3,\n### Instruction:\nਜਾਂਚ ਕਰੋ ਕਿ ਕੀ ਇਹ ਵਾਕ ਵਿਆਕ...,ਹਾਂ
4,\n### Instruction:\nਦਿੱਤੇ ਪੈਰੇ ਤੋਂ ਇੱਕ ਤੱਥ ਮੁੜ...,ਐਪਲ ਨੇ ਇਸ ਹਫਤੇ Apple M1 ਚਿੱਪ ਦੀ ਘੋਸ਼ਣਾ ਕੀਤੀ.


In [3]:
new_header = ['predictions', 'references']
saved_preds_df.columns = new_header
saved_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   predictions  4284 non-null   object
 1   references   4282 non-null   object
dtypes: object(2)
memory usage: 67.1+ KB


In [4]:
saved_preds_df.head()

,predictions,references
0,\n### Instruction:\n5 ਬਾਹਰੀ ਗਤੀਵਿਧੀਆਂ ਦੀ ਸੂਚੀ ...,1. ਕੁਦਰਤ ਦੀ ਸੈਰ ਕਰੋ।\n2. ਸਾਈਕਲ ਦੀ ਸਵਾਰੀ ਲਈ ਜਾਓ...
1,\n### Instruction:\nਪੰਜ ਗੁਣ ਦੱਸੋ ਜੋ ਕਿਸੇ ਨੂੰ ਪ...,1. ਭਰੋਸੇਯੋਗ\n2. ਉਦਾਰ\n3. ਸਹਾਇਕ\n4. ਗੈਰ-ਨਿਰਣਾਇਕ...
2,\n### Instruction:\nਇੱਕ ਕਾਲਪਨਿਕ ਸੌਫਟਵੇਅਰ ਕੰਪਨੀ...,ਅੰਤਮ ਸੌਫਟਵੇਅਰ ਪੈਕੇਜ ਬਣਾਓ ਅਤੇ 30% ਦੀ ਛੂਟ ਪ੍ਰਾਪਤ...
3,\n### Instruction:\nਜਾਂਚ ਕਰੋ ਕਿ ਕੀ ਇਹ ਵਾਕ ਵਿਆਕ...,ਹਾਂ
4,\n### Instruction:\nਦਿੱਤੇ ਪੈਰੇ ਤੋਂ ਇੱਕ ਤੱਥ ਮੁੜ...,ਐਪਲ ਨੇ ਇਸ ਹਫਤੇ Apple M1 ਚਿੱਪ ਦੀ ਘੋਸ਼ਣਾ ਕੀਤੀ.


In [5]:
saved_preds_df = saved_preds_df.dropna()
saved_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4282 entries, 0 to 4283
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   predictions  4282 non-null   object
 1   references   4282 non-null   object
dtypes: object(2)
memory usage: 100.4+ KB


In [6]:
saved_preds_df.head()

,predictions,references
0,\n### Instruction:\n5 ਬਾਹਰੀ ਗਤੀਵਿਧੀਆਂ ਦੀ ਸੂਚੀ ...,1. ਕੁਦਰਤ ਦੀ ਸੈਰ ਕਰੋ।\n2. ਸਾਈਕਲ ਦੀ ਸਵਾਰੀ ਲਈ ਜਾਓ...
1,\n### Instruction:\nਪੰਜ ਗੁਣ ਦੱਸੋ ਜੋ ਕਿਸੇ ਨੂੰ ਪ...,1. ਭਰੋਸੇਯੋਗ\n2. ਉਦਾਰ\n3. ਸਹਾਇਕ\n4. ਗੈਰ-ਨਿਰਣਾਇਕ...
2,\n### Instruction:\nਇੱਕ ਕਾਲਪਨਿਕ ਸੌਫਟਵੇਅਰ ਕੰਪਨੀ...,ਅੰਤਮ ਸੌਫਟਵੇਅਰ ਪੈਕੇਜ ਬਣਾਓ ਅਤੇ 30% ਦੀ ਛੂਟ ਪ੍ਰਾਪਤ...
3,\n### Instruction:\nਜਾਂਚ ਕਰੋ ਕਿ ਕੀ ਇਹ ਵਾਕ ਵਿਆਕ...,ਹਾਂ
4,\n### Instruction:\nਦਿੱਤੇ ਪੈਰੇ ਤੋਂ ਇੱਕ ਤੱਥ ਮੁੜ...,ਐਪਲ ਨੇ ਇਸ ਹਫਤੇ Apple M1 ਚਿੱਪ ਦੀ ਘੋਸ਼ਣਾ ਕੀਤੀ.


In [7]:
saved_preds_df.to_csv("Predictions_test_data.csv", index=None)

In [24]:
predictions=list(saved_preds_df['predictions'])
references=list(saved_preds_df['references'])

In [26]:
# BLEU
bleu = load("bleu")
bleu_results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU Score: {bleu_results['bleu']}")


BLEU Score: 0.03157486421394274


In [30]:
# ROUGE
rouge = load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE Score: {rouge_results}")  # This will print different ROUGE variants

ROUGE Score: {'rouge1': 0.038499762388349804, 'rouge2': 0.013019296072536182, 'rougeL': 0.03762116227346678, 'rougeLsum': 0.037291436269988584}


In [42]:
# METEOR (hold for now)
#meteor = load("meteor")
#meteor_results = meteor.compute(predictions=predictions, references=references)
#print(f"METEOR Score: {meteor_results['meteor']}")

In [ ]:
# BERTScore
bertscore = load("bertscore")
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(f"BERTScore: {bertscore_results}")  # This will print precision, recall, and F1

In [20]:
#!pip install -q mauve-text

In [ ]:
# Dialogue Generation Metrics (Example - Distinct-1 and Distinct-2)
def distinct_n(texts, n):
    """Calculates the number of distinct n-grams divided by the total number of n-grams."""
    all_ngrams = []
    for text in texts:
        tokens = nltk.word_tokenize(text)
        ngrams = list(nltk.ngrams(tokens, n))
        all_ngrams.extend(ngrams)
    return len(set(all_ngrams)) / len(all_ngrams) if all_ngrams else 0

distinct_1_score = distinct_n(predictions, 1)
distinct_2_score = distinct_n(predictions, 2)
print(f"Distinct-1: {distinct_1_score}")
print(f"Distinct-2: {distinct_2_score}")

# MAUVE (This requires the MAUVE package - pip install mauve-text)
try:
    from mauve import compute_mauve
    mauve_results = compute_mauve(p_text=predictions, q_text=references)
    print(f"MAUVE Score: {mauve_results}")
except ImportError:
    print("MAUVE package not found. Please install it with 'pip install mauve-text'")

In [25]:
# Self-BLEU 1
# Self-BLEU (This requires a bit more complex logic to compare generated texts to each other)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def self_bleu(texts):
    """Calculates the average self-BLEU score of each generated text against all other generated texts."""
    self_bleu_scores = []
    for i in range(len(texts)):
        current_text = texts[i].split()  # Tokenize the current text
        other_texts = [texts[j].split() for j in range(len(texts)) if j != i]  # Tokenize other texts
        bleu_score = sentence_bleu(other_texts, current_text, smoothing_function=SmoothingFunction().method1)
        self_bleu_scores.append(bleu_score)
    return sum(self_bleu_scores) / len(self_bleu_scores)

self_bleu_score = self_bleu(predictions)
print(f"Self-BLEU Score: {self_bleu_score}")

Self-BLEU Score: 0.4602623690010304


In [ ]:
# Self-BLEU 2
# Self-BLEU (This requires a bit more complex logic to compare generated texts to each other)
def self_bleu(texts):
    """Calculates the average BLEU score of each generated text against all other generated texts."""
    self_bleu_scores = []
    for i in range(len(texts)):
        # Calculate BLEU for the current text against all other texts
        current_text = texts[i]
        other_texts = texts[:i] + texts[i+1:]
        bleu_result = bleu.compute(predictions=[current_text], references=[other_texts])
        self_bleu_scores.append(bleu_result['bleu'])
    return sum(self_bleu_scores) / len(self_bleu_scores)

self_bleu_score = self_bleu(predictions)
print(f"Self-BLEU Score: {self_bleu_score}") 
